In [2]:
!git clone https://github.com/lucas-battisti/custom-pytorch-lightning-module.git
!mv "custom-pytorch-lightning-module/modules.py" "modules.py"
!rm -rf "custom-pytorch-lightning-module"



!git clone https://github.com/lucas-battisti/pytorch-blocks.git
!mv "pytorch-blocks/blocks.py" "blocks.py"
!rm -rf "pytorch-blocks"

Cloning into 'custom-pytorch-lightning-module'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 46 (delta 25), reused 28 (delta 13), pack-reused 0
Receiving objects: 100% (46/46), 22.47 KiB | 442.00 KiB/s, done.
Resolving deltas: 100% (25/25), done.
Cloning into 'pytorch-blocks'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 29 (delta 15), reused 18 (delta 8), pack-reused 0
Receiving objects: 100% (29/29), 20.19 KiB | 1.12 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import dt
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/galax.csv').drop('ID', axis=1).astype('float32')

In [4]:
df.iloc[:, 2:4]
a = [34, 35] + list(range(5, 28, 2)) + [30, 31]
b = [36, 37] + list(range(6, 29, 2)) + [32, 33]
df.iloc[:, b]

,e_FUVmag,e_NUVmag,e_u_PStotal,e_J0378_PStotal,e_J0395_PStotal,e_J0410_PStotal,e_J0430_PStotal,e_g_PStotal,e_J0515_PStotal,e_r_PStotal,e_J0660_PStotal,e_i_PStotal,e_J0861_PStotal,e_z_PStotal,e_W1_MAG,e_W2_MAG
0,NaN,NaN,0.224955,0.311987,0.518295,0.194772,0.245713,0.041893,0.091243,0.025094,0.029657,0.023942,0.055467,0.036479,0.033687,0.083541
1,NaN,NaN,0.452150,0.671358,0.382968,1.782604,0.585214,0.124910,0.304120,0.064142,0.059612,0.048313,0.102840,0.068152,0.025455,0.062376
2,NaN,NaN,NaN,NaN,0.853746,24.218925,2.501554,0.662034,0.650283,0.137526,0.152992,0.092161,0.250096,0.123786,0.019780,0.081629
3,NaN,NaN,2.673750,0.911359,0.611246,0.835895,0.634541,0.132246,0.293990,0.064867,0.061165,0.047712,0.101811,0.062711,0.027340,0.058215
4,NaN,NaN,0.727081,0.447041,1.601313,0.658014,0.204889,0.070558,0.167561,0.042762,0.063853,0.035506,0.072106,0.049442,0.020226,0.054294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324068,0.4851,0.1726,2.584847,1.625167,2.574174,0.459246,0.648415,0.106416,0.284852,0.066846,0.064942,0.065221,0.119365,0.117191,NaN,NaN
324069,0.1047,0.0587,0.168559,0.303243,0.330396,0.390139,0.248238,0.069870,0.143726,0.037100,0.040685,0.031847,0.057521,0.044997,0.012762,0.044740
324070,NaN,NaN,1.007946,0.881726,NaN,1.336982,1.126299,0.116354,0.181017,0.036185,0.028266,0.018727,0.026390,0.021848,0.009996,0.021091
324071,NaN,NaN,NaN,6.650383,NaN,0.959220,NaN,0.318630,0.995016,0.114878,0.110119,0.081381,0.182333,0.112433,0.054559,0.129773


In [4]:
set_size = (np.round(np.array([0.5, 0.25, 0.25])*df.shape[0]) + np.array([1, 0, 0])).astype(int).tolist()
a = [34, 35] + list(range(5, 28, 2)) + [30, 31]
b = [36, 37] + list(range(6, 29, 2)) + [32, 33]

f = df.iloc[:, a]
e_f = df.iloc[:, b]
extra = df.iloc[:, 2:4]

z = df.iloc[:, 38]

In [5]:
import lightning as L

from modules import RegressionModule as Reg
from blocks import LinearBlock, Conv1dBlock, Conv2dBlock
from data import FF_Dataset, Custom_DataModule, CNN1D_Dataset, CNN2D_Dataset

import torch
from torch import nn
from torch.nn import MaxPool1d, AvgPool1d
from torch.optim.lr_scheduler import ReduceLROnPlateau

from multi import Slicer

from math import floor

torch.cuda.is_available()

True

In [6]:
dm = {}
m = {}

# FF with filters and no errors

In [6]:
str = 'ff_filters'

dm[str] = Custom_DataModule(xez = (f, None, z), extra=extra,
                       dataset_class=FF_Dataset,
                       norm=True,
                       split_size_or_sections=set_size,
                       batch_size=2000, num_workers=7)

l1 = LinearBlock(in_features=18, out_features=12,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 12},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=12, out_features=12,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 12},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=12, out_features=1)

sequential = nn.Sequential(l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=2,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

NameError: name 'dm' is not defined

# FF with filters and errors

In [8]:
str = 'ff_filters_errors'

dm[str] = Custom_DataModule(xez = (f, e_f, z), extra=extra,
                       dataset_class=FF_Dataset,
                       norm=True,
                       split_size_or_sections=set_size,
                       batch_size=2000, num_workers=7)

l1 = LinearBlock(in_features=34, out_features=24,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 24},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=24, out_features=24,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 24},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=24, out_features=1)

sequential = nn.Sequential(l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=2,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-05-19 17:20:15.721040: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-19 17:20:15.759343: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 17:20:16.741041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 1.6 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
1.6 K   

[{}]

# CNN1D_no with filters

In [ ]:
str = 'cnn1d_no_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z), extra=extra,
                       dataset_class=CNN1D_Dataset,
                       version='no', norm=True,
                       set_size=set_size,
                       batch_size=2000, num_workers=7)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=192, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 53.9 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
53.9 K    Trainable params
0         Non-trainable params
53.9 K    Total params
0.215     Total estimated model params size (MB)
INFO:lightning.

[{}]

In [38]:
import importlib
import multi
importlib.reload(multi)

<module 'multi' from '/home/lucas-battisti/Área de Trabalho/python-projects/Acnodnem-Ailiram/multi.py'>

In [42]:
str = 'cnn1d_no_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z), extra=extra,
                       dataset_class=CNN1D_Dataset,
                       version='no', norm=True,
                       split_size_or_sections=set_size,
                       batch_size=2000, num_workers=7)

sl = multi.Slicer(2, dim = 2)

sl_ = sl(dm['cnn1d_no_filters'].train[1:3][0])


c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

l1 = LinearBlock(in_features=194, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)



multimodel1 = multi.MultiModule(c1, nn.Identity())
multimodel2 = multi.MultiModule(c2, nn.Identity())

multimodel1_ = multimodel1(sl_)
multimodel2_ = multimodel2(multimodel1_)

fl = multi.Joiner_Flatten()

fl_ = fl(multimodel2_)

sequential = multi.MultiInputSequential(sl, multimodel1, multimodel2, fl, l1, l2, l3)

sequential(dm['cnn1d_no_filters'].train[1:3][0]).size()


torch.Size([2, 1])

# CNN1D_no with diffs

In [ ]:
str = 'cnn1d_no_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN1D_Dataset,
                       version='no', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=128, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 44.7 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
44.7 K    Trainable params
0         Non-trainable params
44.7 K    Total params
0.179     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_with with filters

In [ ]:
str = 'cnn1d_with_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN1D_Dataset,
                       version='with', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=448, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=336, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=336, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 270 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
270 K     Trainable params
0         Non-trainable params
270 K     Total params
1.081     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_with with diffs

In [ ]:
str = 'cnn1d_with_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN1D_Dataset,
                       version='with', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=384, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=336, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=336, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,max_epochs=250
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 248 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
248 K     Trainable params
0         Non-trainable params
248 K     Total params
0.995     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_stack with filters

In [ ]:
str = 'cnn1d_stack_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN1D_Dataset,
                       version='stack', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=2, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=192, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 54.0 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
54.0 K    Trainable params
0         Non-trainable params
54.0 K    Total params
0.216     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_stack with diffs

In [ ]:
str = 'cnn1d_stack_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN1D_Dataset,
                       version='stack', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=2, out_channels=32,
                 kernel_size=4,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=128, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 46.9 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
46.9 K    Trainable params
0         Non-trainable params
46.9 K    Total params
0.187     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN2D with filters

In [ ]:
str = 'cnn2d_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN2D_Dataset,
                       K=20, t_inf=10, t_sup=10, norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv2dBlock(in_channels=1, out_channels=32,
                 kernel_size=4,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv2dBlock(in_channels=32, out_channels=64,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=384, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=288, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=288, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=175,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 214 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
214 K     Trainable params
0         Non-trainable params
214 K     Total params
0.859     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module |

[{}]

# CNN2D with diffs

In [ ]:
str = 'cnn2d_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN2D_Dataset,
                       K=20, t_inf=10, t_sup=10, norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv2dBlock(in_channels=1, out_channels=32,
                 kernel_size=5,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv2dBlock(in_channels=32, out_channels=64,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=192, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=288, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=288, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 159 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.639     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module |

[{}]

In [ ]:
!zip -r ff_filters.zip ff_filters
#!zip -r ff_filters_errors.zip ff_filters_errors
#!zip -r ff_diffs.zip ff_diffs
#!zip -r ff_diffs_errors.zip ff_diffs_errors
#!zip -r cnn1d_no_filters.zip cnn1d_no_filters
#!zip -r cnn1d_no_diffs.zip cnn1d_no_diffs
#!zip -r cnn1d_with_filters.zip cnn1d_with_filters
#!zip -r cnn1d_with_diffs.zip cnn1d_with_diffs
#!zip -r cnn1d_stack_filters.zip cnn1d_stack_filters
#!zip -r cnn1d_stack_diffs.zip cnn1d_stack_diffs
!zip -r cnn2d_filters.zip cnn2d_filters
!zip -r cnn2d_diffs.zip cnn2d_diffs

  adding: ff_filters/ (stored 0%)
  adding: ff_filters/lightning_logs/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_std/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_std/events.out.tfevents.1712597390.605934e206d9.368.26 (deflated 10%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (train)_mean/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (train)_mean/events.out.tfevents.1712597079.605934e206d9.368.17 (deflated 74%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_size/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_size/events.out.tfevents.1712597390.605934e206d9.368.27 (deflated 10%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (validation)_median/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/s

In [ ]:
from google.colab import files

files.download("ff_filters.zip")
#files.download("ff_filters_errors.zip")
#files.download("ff_diffs.zip")
#files.download("ff_diffs_errors.zip")
#files.download("cnn1d_no_filters.zip")
#files.download("cnn1d_no_diffs.zip")
#files.download("cnn1d_with_filters.zip")
#files.download("cnn1d_with_diffs.zip")
#files.download("cnn1d_stack_filters.zip")
#files.download("cnn1d_stack_diffs.zip")
files.download("cnn2d_filters.zip")
files.download("cnn2d_diffs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#!unzip ff_filters_errors.zip
!unzip ff_diffs.zip
!unzip ff_diffs_errors.zip
!unzip cnn1d_no_filters.zip
!unzip cnn1d_no_diffs.zip
!unzip cnn1d_with_filters.zip
!unzip cnn1d_with_diffs.zip
!unzip cnn1d_stack_filters.zip
!unzip cnn1d_stack_diffs.zip

Archive:  ff_diffs.zip
   creating: ff_diffs/
   creating: ff_diffs/lightning_logs/
   creating: ff_diffs/lightning_logs/version_0/
   creating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_median/
  inflating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_median/events.out.tfevents.1712571718.5987425793bc.2443.70  
   creating: ff_diffs/lightning_logs/version_0/losses_training_loss/
  inflating: ff_diffs/lightning_logs/version_0/losses_training_loss/events.out.tfevents.1712571718.5987425793bc.2443.67  
   creating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_max/
  inflating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_max/events.out.tfevents.1712571718.5987425793bc.2443.72  
   creating: ff_diffs/lightning_logs/version_0/squared_errors_stats (validation)_std/
  inflating: ff_diffs/lightning_logs/version_0/squared_errors_stats (validation)_std/events.out.tfevents.1712571717.5987425793bc.2443.65  
   creating: ff_